# Execution of <span style="color:red">Win Sports Journey</span> project

In [1]:
# Change directory to workspace root so that relative path loads work correctly.
import os
try:
    os.chdir(os.path.join(os.getcwd(), '../'))
    print(os.getcwd())
except:
    pass
from ia.jira import connect, issue as ticket
from confluence.client import Confluence
from ia.confluence import content, page
from ia.algo import dependency, project
import datetime
from IPython.display import Image
from IPython.display import Markdown as md

report_start_time = datetime.datetime.now()
report_timestamp = f'`Report executed at {report_start_time:%Y-%m-%d %H:%M}`'
md(report_timestamp)

/Users/lswolkien/Devel/jiraissueanalyzer


`Report executed at 2020-06-04 08:51`

# Report variables

In [2]:
confluence_space_key = "~lswolkien"
confluence_parent_page = "Reports"
confluence_page_title = "Win Sports Journey execution"
epic_key = "DANMR-609"
product_name = "Win Sports Journey"
report_title = f'<span style="color:red">{product_name}</span> project execution dashboard'
report_description = f"Project Status Report: {report_start_time:%Y-%m-%d}"



# Access variables

In [3]:
jira_url = os.environ['JIRA_URL']
jira_username = os.environ['JIRA_USERNAME']
jira_password = os.environ['JIRA_PASSWORD']

conf_url = os.environ['CONFLUENCE_URL']
conf_username = os.environ['CONFLUENCE_USERNAME']
conf_password = os.environ['CONFLUENCE_PASSWORD']

## Calculate metrics based on Jira data

In [4]:
jira = connect(jira_url, basic_auth=(jira_username, jira_password))

jql = f'"Epic Link" = {epic_key} and issuetype not in ("Test")'
status_done=('Done', 'Waiting for production')

percentage_done, all_issues, done_issues = project.progress(jira, jql, status_done)

## Publish the report on the Confluence page

In [5]:
with Confluence(conf_url, (conf_username, conf_password)) as c:
    page_builder = page.PageBuilder(c)
    new_attachments = []

    new_content = page.format_text("h2", report_title)
    new_content += page.format_text("p", report_description)
    new_content += page.format_text("h4", "Project related links:")
    new_content += page.format_text("p", f'<a href="{jira_url}/browse/{epic_key}">Win Sports Journey MVP business description</a>')
    new_content += page.format_text("p", f'<a href="{conf_url}/display/SMDP/Onsite+Messages+with+Optimizely">Onsite Messages with Optimizely SolDef</a>')
    new_content += page.format_text("h4", "Other usefull links:")
    new_content += page.format_text("p", f'<a href="{conf_url}/pages/viewpage.action?pageId=359924452">Workflow & Statuses definition</a>')
    new_content += page.format_text("p", f'<a href="{conf_url}/display/SMDP/Jira+issue+links+type+definition">How to correctly link dependencies</a>')
   
        
    project_content, project_attachments = content.project_report(percentage_done, all_issues, jql, status_done, product_name, "Retention - Sprint board")
    new_content += project_content
    new_attachments += project_attachments

    # embed page with project notes
    new_content += page.embed_conf_page(f'{product_name} Project Issues')

    page_builder.create_or_update(
        space_key=confluence_space_key, 
        title=confluence_page_title, 
        new_content=new_content, 
        attachments=new_attachments,
        parent_page_title=confluence_parent_page
    )
    

In [6]:
report_end_time = datetime.datetime.now()
report_timestamp = f'` Report finished at {report_end_time:%Y-%m-%d %H:%M}, time elapsed: {report_end_time - report_start_time}`'
md(report_timestamp)

` Report finished at 2020-06-04 08:52, time elapsed: 0:00:27.239419`

In [7]:
import glob
files = glob.glob('*.png')
for filename in files:
    try:
        os.remove(filename)
    except:
        print("Error while deleting file : ", filename)